In [25]:
from confluent_kafka import Producer
from random import choice

producer = Producer({'bootstrap.servers': 'localhost:9092'})

#build delivery callback
def delivery_callback(err, msg):
    if err:
        print("Error occured: {}".format(err))
    else:
        print("Produced event to topic {topic}".format(topic = msg.topic()))

topic = "purchase"
user_names = ['T', 'Jame', 'Smith']
products = ['Food', 'Suit', 'Bike', 'Phone']

for _ in range(10):
    user_name = choice(user_names)
    product = choice(products)
    producer.produce(topic, product, user_name, callback = delivery_callback)

producer.poll(1000)
producer.flush()


Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase
Produced event to topic purchase


0

In [26]:
from confluent_kafka import Consumer

consumer = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'product_group',
    'auto.offset.reset': 'earliest'
})

consumer.subscribe(['purchase'])

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            print("Waiting... ")
        
        elif msg.error():
            print("Error occured: {}".format(msg.error()))
        else:
            print("Consumed event from topic {topic}: key = {key} value = {value}" \
                    .format(topic = msg.topic(), key = msg.key().decode('utf-8'), value = msg.value().decode('utf-8')))
except KeyboardInterrupt:
        pass
finally:
    consumer.close()

Waiting... 
Waiting... 
Waiting... 
Consumed event from topic purchase: key = Smith value = Bike
Consumed event from topic purchase: key = Jame value = Food
Consumed event from topic purchase: key = Jame value = Food
Consumed event from topic purchase: key = T value = Suit
Consumed event from topic purchase: key = T value = Phone
Consumed event from topic purchase: key = T value = Food
Consumed event from topic purchase: key = Jame value = Phone
Consumed event from topic purchase: key = T value = Bike
Consumed event from topic purchase: key = Smith value = Bike
Consumed event from topic purchase: key = Jame value = Suit
Waiting... 
Waiting... 
Waiting... 
Waiting... 
Waiting... 
Waiting... 


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

spark = SparkSession.builder \
                    .appName("Test") \
                    .getOrCreate()

#schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("value", StringType(), True)
])

df = spark.readStream \
          .format("csv") \
          .schema(schema) \
          .load("./data")

df = df.select('value')

query = df.writeStream \
          .outputMode("append") \
          .format("console") \
          .start()

query.awaitTermination()

Py4JJavaError: An error occurred while calling o33.load.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$